In [4]:
from collections import Counter
import pandas as pd
import datetime as dt
from flask import Flask, render_template, redirect,jsonify
from flask_pymongo import PyMongo
import pymongo
import csv
import json
import pandas as pd
import requests
import json



response = requests.get('http://127.0.0.1:5000/netflixdata').json()
# print(data)

df = pd.DataFrame.from_dict(pd.json_normalize(response), orient='columns')

df=df.drop('show_id', axis=1)
df

,cast,country,date_added,description,director,duration,listed_in,rating,release_year,title,type
0,na,United States,"September 25, 2021","As her father nears the end of his life, filmm...",Kirsten Johnson,90 min,Documentaries,PG-13,2020,Dick Johnson Is Dead,Movie
1,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021","After crossing paths at a party, a Cape Town t...",na,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries",TV-MA,2021,Blood & Water,TV Show
2,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",na,"September 24, 2021",To protect his family from a powerful drug lor...,Julien Leclercq,1 Season,"Crime TV Shows, International TV Shows, TV Act...",TV-MA,2021,Ganglands,TV Show
3,na,na,"September 24, 2021","Feuds, flirtations and toilet talk go down amo...",na,1 Season,"Docuseries, Reality TV",TV-MA,2021,Jailbirds New Orleans,TV Show
4,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",In a city of coaching centers known to train I...,na,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",TV-MA,2021,Kota Factory,TV Show
...,...,...,...,...,...,...,...,...,...,...,...
8802,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019","A political cartoonist, a crime reporter and a...",David Fincher,158 min,"Cult Movies, Dramas, Thrillers",R,2007,Zodiac,Movie
8803,na,na,"July 1, 2019","While living alone in a spooky town, a young g...",na,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies",TV-Y7,2018,Zombie Dumb,TV Show
8804,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",Looking to survive in a world taken over by zo...,Ruben Fleischer,88 min,"Comedies, Horror Movies",R,2009,Zombieland,Movie
8805,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020","Dragged from civilian life, a former superhero...",Peter Hewitt,88 min,"Children & Family Movies, Comedies",PG,2006,Zoom,Movie


In [5]:
description_list=df.description

In [6]:
description_list

0       As her father nears the end of his life, filmm...
1       After crossing paths at a party, a Cape Town t...
2       To protect his family from a powerful drug lor...
3       Feuds, flirtations and toilet talk go down amo...
4       In a city of coaching centers known to train I...
                              ...                        
8802    A political cartoonist, a crime reporter and a...
8803    While living alone in a spooky town, a young g...
8804    Looking to survive in a world taken over by zo...
8805    Dragged from civilian life, a former superhero...
8806    A scrappy but poor boy worms his way into a ty...
Name: description, Length: 8807, dtype: object

In [7]:
# import csv

# stopwords=[]

# with open('../data/stopwords.csv', newline='') as f:
#     reader = csv.reader(f)
#     data = list(reader)

response = requests.get('http://127.0.0.1:5000/stopwords').json()
# print(response)
stopwords=[]
for dic in response:
    for key,value in dic.items():
        stopwords.append(value)

print(stopwords)


# stopwords_df = pd.DataFrame.from_dict(pd.json_normalize(response), orient='columns')


# for word in data:
#     stopwords.append(word[0])



['a', 'able', 'about', '‚Äî', '‚Äì', 'above', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'added', 'adj', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'again', 'against', 'ah', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apparently', 'approximately', 'are', 'aren', 'arent', 'arise', 'around', 'as', 'aside', 'ask', 'asking', 'at', 'auth', 'available', 'away', 'awfully', 'b', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'begin', 'beginning', 'beginnings', 'begins', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'between', 'beyond', 'biol', 'both', 'brief', 'briefly', 'but', 'by', 'c', 'ca', 'came', 'can', 'cannot', "can't", 'cause', 'causes', 'certain', 'certainly', 'co', 'com', 'come', 

In [8]:
word_list=[]
word_frequency={}

for entry in description_list:
    word_list=entry.split()
#     print(word_list)
    
    for key in word_list:
        if key.lower() not in stopwords:
            if key not in word_frequency:
                word_frequency[key] =1

            else:
                word_frequency[key] += 1
        else:
            pass
    word_list=[]
    


# word_frequency['value'] = word_frequency.pop('141')
print(word_frequency)



# with open('word_frequency.csv', 'w') as f:
#     for key in word_frequency.keys():
#         f.write("%s,%s\n"%(key,word_frequency[key]))

    

{'father': 160, 'nears': 3, 'life,': 67, 'filmmaker': 45, 'Kirsten': 1, 'Johnson': 2, 'stages': 13, 'death': 119, 'inventive': 5, 'comical': 5, 'ways': 34, 'help': 266, 'face': 106, 'inevitable.': 1, 'crossing': 4, 'paths': 48, 'party,': 17, 'Cape': 4, 'Town': 4, 'teen': 196, 'prove': 56, 'private-school': 2, 'swimming': 4, 'star': 105, 'sister': 58, 'abducted': 13, 'birth.': 4, 'protect': 71, 'family': 446, 'powerful': 87, 'drug': 112, 'lord,': 2, 'skilled': 18, 'thief': 27, 'Mehdi': 2, 'expert': 32, 'team': 173, 'robbers': 8, 'pulled': 6, 'violent': 62, 'deadly': 67, 'turf': 3, 'war.': 9, 'Feuds,': 1, 'flirtations': 2, 'toilet': 2, 'talk': 23, 'incarcerated': 6, 'women': 132, 'Orleans': 9, 'Justice': 2, 'Center': 5, 'gritty': 15, 'reality': 86, 'series.': 67, 'city': 88, 'coaching': 5, 'centers': 11, 'train': 28, 'India‚Äôs': 14, 'finest': 2, 'collegiate': 1, 'minds,': 2, 'earnest': 4, 'unexceptional': 2, 'student': 131, 'friends': 359, 'navigate': 66, 'campus': 8, 'arrival': 14, 'ch

In [9]:
data_items = word_frequency.items()
data_list = list(data_items)
df = pd. DataFrame(data_list) 
print(df)

                 0    1
0           father  160
1            nears    3
2            life,   67
3        filmmaker   45
4          Kirsten    1
...            ...  ...
29550      sadist.    1
29551  Francisco's    1
29552        dorky    1
29553    roughneck    1
29554      Dragged    1

[29555 rows x 2 columns]


In [10]:
df.columns=['word','value']

In [11]:
df

,word,value
0,father,160
1,nears,3
2,"life,",67
3,filmmaker,45
4,Kirsten,1
...,...,...
29550,sadist.,1
29551,Francisco's,1
29552,dorky,1
29553,roughneck,1


In [12]:
df=df.sort_values(by='value', ascending=False)

In [13]:
df=df.drop([551,95])

In [14]:
df_30=df.head(30).to_csv('word_frequency30.csv')
data_30=df.head(30).to_dict(orient='records')
# import data into mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db=client['Netflix']
col1=db.top30
if col1.drop():
    print('Deleted')
else:
    col1=db.top30

col1.insert_many(data_30)

In [15]:
# df_50=df.head(50).to_csv('word_frequency50.csv')
data_50=df.head(50).to_dict(orient='records')
col2=db.top50
if col2.drop():
    print('Deleted')
else:
    col2=db.top50
col2.insert_many(data_50)

In [16]:
# df_100=df.head(100).to_csv('word_frequency100.csv')
data_100=df.head(100).to_dict(orient='records')
col3=db.top100
if col3.drop():
    print('Deleted')
else:
    col3=db.top100
col3.insert_many(data_100)